Data Exploration, what does the data look like?

In [1]:
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed
import requests
from pathlib import Path
from tqdm import tqdm
from zipfile import ZipFile
import pandas as pd
import numpy as np

_STATE_CODES = {
    'AL': '01', 'AK': '02', 'AZ': '04', 'AR': '05', 'CA': '06',
    'CO': '08', 'CT': '09', 'DE': '10', 'FL': '12', 'GA': '13',
    'HI': '15', 'ID': '16', 'IL': '17', 'IN': '18', 'IA': '19',
    'KS': '20', 'KY': '21', 'LA': '22', 'ME': '23', 'MD': '24',
    'MA': '25', 'MI': '26', 'MN': '27', 'MS': '28', 'MO': '29',
    'MT': '30', 'NE': '31', 'NV': '32', 'NH': '33', 'NJ': '34',
    'NM': '35', 'NY': '36', 'NC': '37', 'ND': '38', 'OH': '39',
    'OK': '40', 'OR': '41', 'PA': '42', 'RI': '44', 'SC': '45',
    'SD': '46', 'TN': '47', 'TX': '48', 'UT': '49', 'VT': '50',
    'VA': '51', 'WA': '53', 'WV': '54', 'WI': '55', 'WY': '56',
    # 'PR': '72'  # Puerto Rico
}

acs_dtypes = {
    'PINCP': float, 'RT': str, 'SOCP': str, 'SERIALNO': str, 'NAICSP': str, 'AGEP': int, 'COW': float, 'SCHL': float,
    'MAR': int, 'OCCP': float, 'POBP': int, 'RELP': int, 'RELSHIPP': int, 'WKHP': int, 'SEX': int, 'RAC1P': int
}


def get_csv_file(state, year, horizon, survey):
    state_code = _STATE_CODES[state]
    survey_code = 'p' if survey == 'person' else 'h'
    if int(year) >= 2017:
        file_name = f'psam_{survey_code}{state_code}.csv'
    else:
        file_name = f'ss{str(year)[-2:]}{survey_code}{state.lower()}.csv'

    data_dir = Path(f'./data/{year}/{horizon}')
    file_path = data_dir / file_name
    return file_path


def download_acs_csv(state, year, horizon, survey):
    file_path = get_csv_file(state, year, horizon, survey)
    if file_path.is_file():
        return file_path

    data_dir = file_path.parent
    survey_code = 'p' if survey == 'person' else 'h'

    if int(year) == 2020 and horizon == '1-Year':
        base_url = f'https://www2.census.gov/programs-surveys/acs/experimental/{year}/data/pums/{horizon}/'
    else:
        base_url = f'https://www2.census.gov/programs-surveys/acs/data/pums/{year}/{horizon}'

    remote_fname = f'csv_{survey_code}{state.lower()}.zip'
    url = f'{base_url}/{remote_fname}'
    zip_path = data_dir / remote_fname
    try:
        resp = requests.get(url)
        resp.raise_for_status()
        zip_path.write_bytes(resp.content)
        with ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extract(file_path.name, data_dir)
        zip_path.unlink()
    except Exception as e:
        print(f'\nError downloading {year}-{state}: {e}')
    return file_path


def prepare_downloads():
    data_threads = {}
    horizon = '1-Year'
    survey = 'person'
    data_dir = Path(f'./data')
    data_dir.mkdir(parents=True, exist_ok=True)
    for _, _, files in data_dir.walk():
        for subfile in map(Path, files):
            if subfile.is_file() and subfile.suffix == '.zip':
                subfile.unlink()
    with ThreadPoolExecutor() as pool:
        for year in range(2014, 2023):
            year_dir = data_dir / str(year) / horizon
            year_dir.mkdir(parents=True, exist_ok=True)
            for state in _STATE_CODES.keys():
                data_threads[(year, state)] = pool.submit(download_acs_csv, state, year, horizon, survey)
        for future in tqdm(as_completed(data_threads.values()), total=len(data_threads)):
            future.result()
    del data_threads


def adult_filter(data: pd.DataFrame):
    df = data
    df = df[df['AGEP'] > 16]
    df = df[df['PINCP'] > 100]
    df = df[df['WKHP'] > 0]
    df = df[df['PWGTP'] >= 1]
    return df


def load_acs(year, state, horizon='1-Year', survey='person'):
    # print(f"Loading {year}-{state}")
    csv_file = download_acs_csv(state, year, horizon, survey)
    df = pd.read_csv(csv_file)
    df = df.replace(' ', '')
    for col, col_type in acs_dtypes.items():
        if col not in df.columns:
            continue
        if col_type == np.float64:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        elif col_type == int:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(-1).astype(int)
    df['PINCP'] = df['PINCP'].replace('', np.nan).astype(acs_dtypes['PINCP'])
    df = df.astype({k: v for k, v in acs_dtypes.items() if k in df.columns})
    df['RAC1P'] = df['RAC1P'].apply(lambda x: int(x == 1))
    return df


for _ in range(3):
    prepare_downloads()

100%|██████████| 450/450 [00:00<00:00, 450677.36it/s]


In [2]:
def acs_to_income_df(acs_df, salary_threshold=50000):
    features = [
        'AGEP',
        'COW',
        'SCHL',
        'MAR',
        'OCCP',
        'POBP',
        'RELP',
        'RELSHIPP',
        'WKHP',
        'SEX',
        'RAC1P',
    ]
    df = adult_filter(acs_df)
    variables = df[[f for f in features if f in df.columns]].fillna(-1)
    target = pd.DataFrame(df['PINCP'] > salary_threshold).astype(int).reset_index(drop=True)
    return variables, target


def get_all_income_data_by_year(year) -> tuple[pd.DataFrame, pd.DataFrame]:
    data_path = Path(f'./years/{year}/1-Year-Income.csv.gz')
    if data_path.exists():
        acs_income_data = pd.read_csv(data_path, compression='gzip')
        filtered_acs_dtypes = {k: v for k, v in acs_dtypes.items() if k in acs_income_data.columns}
        acs_income_data = acs_income_data.astype(filtered_acs_dtypes)
        target = acs_income_data['PINCP']
        features = acs_income_data.drop(columns=['PINCP'])
        return features, target
    acs_income_data = (acs_to_income_df(load_acs(year, state)) for state in _STATE_CODES.keys())
    features, target = map(pd.concat, zip(*acs_income_data))  # type: ignore
    features.reset_index(drop=True, inplace=True)
    target.reset_index(drop=True, inplace=True)
    data_path.parent.mkdir(parents=True, exist_ok=True)
    pd.concat([features, target], axis=1).to_csv(data_path, index=False)
    return features, target



In [3]:
*years, = range(2014, 2023)
for year in years:
    features, labels = get_all_income_data_by_year(year)
    print(f"Year: {year}")
    print(f"Features shape: {features.shape}")
    print(f"Labels shape: {labels.shape}")
    print(features[:10])
    print(labels[:10])

Year: 2014
Features shape: (1577844, 10)
Labels shape: (1577844,)
   AGEP  COW  SCHL  MAR    OCCP  POBP  RELP  WKHP  SEX  RAC1P
0    49  6.0  16.0    1  7210.0     1     0    60    1      1
1    51  1.0  16.0    1  4220.0     1     0    40    1      1
2    53  1.0  16.0    1  7750.0     1     1    40    2      1
3    51  1.0  16.0    1  5610.0    19     0    40    1      1
4    48  5.0  20.0    1  7430.0    47     1    40    2      1
5    49  1.0  16.0    1  8620.0    53     0    45    1      1
6    38  1.0  11.0    1  8410.0     1     1    40    2      1
7    21  1.0  18.0    5   310.0     1     2    40    1      1
8    40  1.0  22.0    1  1020.0     1     0    40    1      1
9    28  3.0  20.0    5  6730.0     1    16    32    1      1
0    0.0
1    0.0
2    0.0
3    0.0
4    1.0
5    0.0
6    0.0
7    0.0
8    1.0
9    0.0
Name: PINCP, dtype: float64
Year: 2015
Features shape: (1594081, 10)
Labels shape: (1594081,)
   AGEP  COW  SCHL  MAR    OCCP  POBP  RELP  WKHP  SEX  RAC1P
0    5

In [4]:
# features, labels = acs_to_income_df(load_acs(2018, "CA"))

features, labels = get_all_income_data_by_year(2018)

# Print the shapes of the arrays
print(f"Features shape: {features.shape}")
print(f"Labels shape: {labels.shape}")
print(features[:10])
print(labels[:10])


def map_race(race_value):
    # return 0 if race_value == 1.0 else 1  # binarize race (0 = white, 1 = non-white)
    return int(race_value != 1.0)


def map_sex(sex_value):
    # return 1 if sex_value == 1 else 0
    return int(sex_value == 1)


features['RAC1P'] = features['RAC1P'].apply(map_race)
features['SEX'] = features['SEX'].apply(map_sex)
print(features[:10])

Features shape: (1655429, 10)
Labels shape: (1655429,)
   AGEP  COW  SCHL  MAR    OCCP  POBP  RELP  WKHP  SEX  RAC1P
0    18  1.0  18.0    5  4720.0    13    17    21    2      0
1    53  5.0  17.0    5  3605.0    18    16    40    1      1
2    41  1.0  16.0    5  7330.0     1    17    40    1      1
3    18  6.0  18.0    5  2722.0     1    17     2    2      1
4    21  5.0  19.0    5  3870.0    12    17    50    1      1
5    37  5.0  16.0    4  9620.0     1    16    35    1      0
6    19  1.0  19.0    5  5400.0     1    17    10    2      1
7    51  1.0  20.0    3  5840.0     1    17    60    2      1
8    18  1.0  18.0    5  4220.0    12    17    12    2      1
9    18  7.0  18.0    5  4600.0     1    17     8    2      1
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
7    0.0
8    0.0
9    0.0
Name: PINCP, dtype: float64
   AGEP  COW  SCHL  MAR    OCCP  POBP  RELP  WKHP  SEX  RAC1P
0    18  1.0  18.0    5  4720.0    13    17    21    0      1
1    53  5.0  17.0   

In [ ]:
import matplotlib.pyplot as plt

race_distribution = features['RAC1P'].value_counts()
sex_distribution = features['SEX'].value_counts()
# Print the distribution
print("Distribution of White vs Non-White:")
print(race_distribution)

# Plot the distribution
race_distribution.plot(kind='bar')
plt.xlabel('Race')
plt.ylabel('Count')
plt.title('Distribution of White (0) vs Non-White (1) 2018')
plt.xticks(ticks=[0, 1], labels=['White', 'Non-White'])
plt.show()

# Plot the distribution
sex_distribution.plot(kind='bar')
plt.xlabel('Sex')
plt.ylabel('Count')
plt.title('Distribution of Male (0) vs Female (1) 2018')
plt.xticks(ticks=[0, 1], labels=['Male', 'Female'])
plt.show()

true_count = np.sum(labels.values)
false_count = len(labels) - true_count

# Create a bar plot
plt.bar(['False', 'True'], [false_count, true_count])

# Add labels and title
plt.xlabel('Label')
plt.ylabel('Count')
plt.title('Distribution of False vs True Labels')

# Show plot
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


def calculate_fairness_metrics(predictions, labels, sensitive_feature):
    # Statistical Parity
    parity = predictions[sensitive_feature == 0].mean() - predictions[sensitive_feature == 1].mean()

    # True Positive Rate (TPR)
    tpr_0 = (predictions[sensitive_feature == 0] & labels[sensitive_feature == 0]).mean() / labels[
        sensitive_feature == 0].mean()
    tpr_1 = (predictions[sensitive_feature == 1] & labels[sensitive_feature == 1]).mean() / labels[
        sensitive_feature == 1].mean()

    # False Positive Rate (FPR)
    fpr_0 = (predictions[sensitive_feature == 0] & ~labels[sensitive_feature == 0]).mean() / (
        ~labels[sensitive_feature == 0]).mean()
    fpr_1 = (predictions[sensitive_feature == 1] & ~labels[sensitive_feature == 1]).mean() / (
        ~labels[sensitive_feature == 1]).mean()

    return parity, tpr_0, tpr_1, fpr_0, fpr_1


def evaluate_predictions(year, state):
    if state == 'XX':
        features, labels = get_all_income_data_by_year(year)
    else:
        features, labels = acs_to_income_df(load_acs(year, state))

    # Apply race and sex binarization
    features['RAC1P'] = features['RAC1P'].apply(map_race)
    features['SEX'] = features['SEX'].apply(map_sex)
    labels = labels.values.astype(bool).ravel()  # Ensuring it's a 1D array

    # Split the data into training and testing sets
    train_features, test_features, train_labels, test_labels = train_test_split(
        features, labels, test_size=0.2, random_state=1)

    # Create and train the logistic regression model
    model = LogisticRegression(max_iter=10000)
    model.fit(train_features, train_labels)

    # Evaluate the model on the test data
    test_predictions = model.predict(test_features)
    accuracy = accuracy_score(test_labels, test_predictions)
    precision = precision_score(test_labels, test_predictions)
    recall = recall_score(test_labels, test_predictions)
    f1 = f1_score(test_labels, test_predictions)

    # Calculate fairness metrics
    sex_parity, sex_tpr_0, sex_tpr_1, sex_fpr_0, sex_fpr_1 = calculate_fairness_metrics(test_predictions,
                                                                                        test_labels,
                                                                                        test_features['SEX'])
    race_parity, race_tpr_0, race_tpr_1, race_fpr_0, race_fpr_1 = calculate_fairness_metrics(test_predictions,
                                                                                             test_labels,
                                                                                             test_features['RAC1P'])
    return {
        'year': year,
        'state': state,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'true_dp': labels.mean(),
        'sex_dp': features['SEX'].mean(),
        'sex_parity': sex_parity,
        'sex_tpr_0': sex_tpr_0,
        'sex_tpr_1': sex_tpr_1,
        'sex_fpr_0': sex_fpr_0,
        'sex_fpr_1': sex_fpr_1,
        'race_dp': features['RAC1P'].mean(),
        'race_parity': race_parity,
        'race_tpr_0': race_tpr_0,
        'race_tpr_1': race_tpr_1,
        'race_fpr_0': race_fpr_0,
        'race_fpr_1': race_fpr_1
    }


*years, = range(2014, 2023)  # From 2014 to 2018
combs = {(year, state) for year in years for state in [*_STATE_CODES.keys(), 'XX']}
results_csv = Path('results.csv.gz')
results_df = pd.DataFrame()
results = []
if results_csv.exists():
    results_df = pd.read_csv(results_csv, compression='gzip')
    existing_combs = set(zip(map(int, results_df['year'].values), results_df['state'].values))
    combs -= existing_combs
    for _, row in results_df.iterrows():
        results.append(row.to_dict())
if combs:
    results_tmp_csv = results_csv.with_suffix('.tmp' + results_csv.suffix)
    for year, state in tqdm(combs):
        results.append(evaluate_predictions(year, state))
        results_df = pd.DataFrame(results)
        if results_csv.exists():
            results_csv.replace(results_tmp_csv)
        results_df.to_csv(results_csv, index=False, compression='gzip')
    if results_tmp_csv.exists():
        results_tmp_csv.unlink()
    
results_df = pd.DataFrame(sorted(results, key=lambda x: (x['year'], x['state'])))
results_df.to_csv(results_csv, index=False, compression='gzip')

# Display the results
print(results_df)


In [ ]:
def plot_fairness_metrics_for_state(state):
    fig, axs = plt.subplots(2, figsize=(10, 12))
    data = results_df[results_df['state'] ==  state]
    dp_ax = axs[0]
    dp_ax.plot(data['year'], data['true_dp'], marker='o', label='$50k+ Percentage')
    dp_ax.plot(data['year'], data['sex_dp'], marker='o', label='Female Percentage')
    dp_ax.plot(data['year'], data['race_dp'], marker='o', label='Non-White Percentage')
    dp_ax.set_xlabel('Year')
    dp_ax.set_ylabel('Percentage')
    dp_ax.set_title('Demographic Percentages Over Time' + (f' for {state}' if state != 'XX' else ''))
    dp_ax.legend()
    mtx_ax = axs[1]
    mtx_ax.plot(data['year'], data['accuracy'], marker='o', label='Accuracy')
    mtx_ax.plot(data['year'], data['precision'], marker='o', label='Precision')
    mtx_ax.plot(data['year'], data['recall'], marker='o', label='Recall')
    mtx_ax.plot(data['year'], data['f1_score'], marker='o', label='F1 Score')
    mtx_ax.set_xlabel('Year')
    mtx_ax.set_ylabel('Score')
    mtx_ax.set_title('Performance Metrics Over Time' + (f' for {state}' if state != 'XX' else ''))
    mtx_ax.legend()    
    plt.show()

plot_fairness_metrics_for_state('XX')

In [ ]:
# Evaluate the difference of the fairness metrics from 2014 to 2022 for each state

metrics = ['accuracy', 'precision', 'recall', 'f1_score']
differences = []
for state in ['XX', *_STATE_CODES.keys()]:
    data = results_df[results_df['state'] == state]
    diff = {'state':state ,**{
        m: data[m].values[-1] - data[m].values[0] for m in metrics        
    }}
    differences.append(diff)
diff_df = pd.DataFrame(differences)

fig, axs = plt.subplots(len(metrics), figsize=(15, 6 * len(metrics)))
for i, metric in enumerate(metrics):
    ax = axs[i]
    data = diff_df[['state',metric]].sort_values(by=metric, ascending=False)
    us = data[data['state'] == 'XX'][metric].values[0]
    ax.axhline(us, color='r', linestyle='--', label='US')
    ax.bar(data['state'], data[metric])
    ax.set_xlabel('State')
    ax.set_ylabel('Difference')
    ax.set_title(f'{metric.capitalize()} Difference from 2014 to 2022')
plt.show()
